本周只有一个代码实践题

以下代码课用来加载 预先训练好的模型,你需要只需要修改模型的存放路径即可（第二行代码）

In [106]:
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from torch.utils.data import DataLoader,Dataset
import numpy as np
import copy

第一步 使用以下链接下载相应预训练模型

https://convaisharables.blob.core.windows.net/lsp/multiref/small_ft.pkl

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
weights = torch.load('data/small_ft.pkl')
medium_config = GPT2Config(n_embd = 768,n_layer = 12, n_head = 12)
model = GPT2LMHeadModel(medium_config)

weights['lm_head.weight'] = weights['lm_head.decoder.weight']
weights.pop('lm_head.decoder.weight',None)

model.load_state_dict(weights)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

你需要写一个推理函数，这个函数接收一个英文句子为输入，输出一个回应。

### 测试语句 
一轮对话   
1. Does money buy happiness ?   
2. What is the best way to buy happiness?   

一轮对话   
1. what is the meaning of a godd life ?   
2. How to be a good person ?   

In [186]:
TOKEN_LEN = tokenizer.vocab_size
def predict(input_words_list):
    # Given previous words, return the probability of the next word.
    input_words = " ".join(input_words_list)
    input_ids = torch.tensor(tokenizer.encode(input_words, add_special_tokens=True)).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_ids=input_ids)
    prediction_score = outputs[0]
    scores = F.softmax(prediction_score[0, -1, :], dim=-1)
    return scores
    
def question_answer(question, beam_size=5):
    inputs_words = question.split(" ") + ['<|endoftext|>']
    
    final_beam_result = []
    
    # The beam search result, 
    # format: [(word_list_1, log_prob_1), (word_list_2, log_prob_2), ... ,(word_list_beam_size, log_prob_beam_size)]
    # Initial beam_result's word list is empty, means all beam searches start with the the same sentence (the input question + '<|endoftext|>')
    beam_result = [([], 0.0)] 
    
    quit = False
    while not quit:
        sum_scores = [] # Save the result of each beam_result
        for result in beam_result:
            scores = predict(inputs_words + result[0])
            sum_scores.append(torch.log(scores) + result[1]) # Use log probability to avoid too small value
        sum_scores = torch.cat(sum_scores) # Size: len(beam_result) * vocab_size
        
        # Find the top beam_size scores from all beam_result
        topk_value, topk_id = torch.topk(sum_scores, beam_size)

        # Update beam_result
        new_beam_result = []
        cnt = 0
        for v, idx in zip(topk_value, topk_id):
            idx_word = idx / TOKEN_LEN # belongs to which beam words list
            idx_item = idx % TOKEN_LEN # word's index

            predicted_words = copy.deepcopy(beam_result[idx_word][0]) # copy words list
            next_word = tokenizer.decode([idx_item]).strip()
            predicted_words.append(next_word)
            new_beam_result.append((predicted_words, v))
        
        # Filter the completed sentence in beam_result, and judge whether to quit
        # First, find the results end with '<|endoftext|>', add them to final_beam_result, and the rest add to beam_result.
        beam_result = []
        for result in new_beam_result:
            if result[0][-1] == '<|endoftext|>':
                final_beam_result.append(result) # stop to generate words, save and wait to output
                beam_size -= 1
            else:
                beam_result.append(result) # continue to generate words
        
        # Then, if there is no result in beam_result, or all results are longer than 100, quit while loop.
        if len(beam_result) == 0:
            quit = True
        for i in beam_result:
            if(len(i[0]) > 100):
                quit = True
    
    # Sort the final_beam_result by the socre which has been normalized with each result's length. 
    answer = sorted(final_beam_result, key = lambda x: x[1] / len(x[0]))
    # Find the max one
    answer = answer[-1]
    
    # Output answer
    result = " ".join(answer[0])
    if (result.endswith('<|endoftext|>')):
        result = result[:-len('<|endoftext|>')]
    print(result)
    

In [190]:
question_answer("Does money buy happiness ?")
question_answer("What is the best way to buy happiness ?")
question_answer("What is the meaning of a good life ?")
question_answer("How to be a good person ?")

Money buys happiness . 
Buy happiness . 
The meaning of life is a good life . 
What ingred iants need to know ? 
